In [1]:
from hdmf.backends.io import HDMFIO
from hdmf.build import BuildManager, GroupBuilder, DatasetBuilder, TypeMap
import pandas as pd
from hdmf.common import ExternalResources, Data
import numpy as np
import json
from hdmf.backends.json import JSONIO
from hdmf.common.resources import KeyTable
from pprint import pprint

In [2]:
er = ExternalResources(name='example')
# Add a species dataset with 2 keys
data1 = Data(
    name='data_name',
    data=np.array(
        [('Mus musculus', 9, 81.0), ('Homo sapiens', 3, 27.0)],
        dtype=[('species', 'U14'), ('age', 'i4'), ('weight', 'f4')]
    )
)
k1, r1, e1 = er.add_ref(
    container=data1,
    field='species',
    key='Mus musculus',
    resource_name='NCBI_Taxonomy',
    resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy',
    entity_id='NCBI:txid10090',
    entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090'
)

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:187: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


In [5]:
path = 'er.json'
with JSONIO(source=path, type_map=er.type_map) as io:
    io.write(er)

In [6]:
with JSONIO(source=path, type_map=er.type_map) as io:
    erd=io.read_to_dict(path='er.json')
    er=io.create_er(name='er', path='er.json')

In [11]:
er

er hdmf.common.resources.ExternalResources at 0x140440258256560
Fields:
  entities: entities <class 'hdmf.common.resources.EntityTable'>
  keys: keys <class 'hdmf.common.resources.KeyTable'>
  object_keys: object_keys <class 'hdmf.common.resources.ObjectKeyTable'>
  objects: objects <class 'hdmf.common.resources.ObjectTable'>
  resources: resources <class 'hdmf.common.resources.ResourceTable'>

In [9]:
# manager= BuildManager(type_map=er.type_map)
# pprint(manager.build(er, root=True))

In [9]:
pprint(erd)

{'data_type': 'ExternalResources',
 'entities': {'attributes': {'data_type': 'Data',
                             'namespace': 'hdmf-common',
                             'object_id': 'ad25c23d-9178-4a01-957e-196914d842c7'},
              'data': [[0,
                        0,
                        'NCBI:txid10090',
                        'https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090']]},
 'keys': {'attributes': {'data_type': 'Data',
                         'namespace': 'hdmf-common',
                         'object_id': '89e765a1-6d0c-4947-ab6b-da0597ec6317'},
          'data': [['Mus musculus']]},
 'namespace': 'hdmf-experimental',
 'object_id': '0df3d714-3f08-4aac-8a00-65a835d93ba2',
 'object_keys': {'attributes': {'data_type': 'Data',
                                'namespace': 'hdmf-common',
                                'object_id': '6d5bd3e9-969a-4eb5-9ab2-ee32b940e1ab'},
                 'data': [[0, 0]]},
 'objects': {'attributes': {'data_type': '

In [15]:
gd={'data_type': 'ExternalResources',
'namespace': 'hdmf-experimental',
'object_id': '0df3d714-3f08-4aac-8a00-65a835d93ba2',}

In [29]:
# GroupBuilder(name='er', groups=gd)

In [36]:
KeyTable().from_dataframe(pd.DataFrame(erd['keys']['data'], columns=KeyTable().columns)).to_dataframe()

,key
0,Mus musculus


In [28]:
KeyTable().columns

('key',)

In [19]:
from hdmf.build import BuildManager, GroupBuilder, DatasetBuilder


In [25]:
from hdmf.backends.hdf5.h5tools import HDF5IO
from hdmf.common import get_manager
path = 'test.nwb'
with HDF5IO(path=path, manager=get_manager(), mode='w') as io:
    io.write(er)

In [29]:
with HDF5IO(path=path, manager=get_manager(), mode='r') as io:
    er_file = io.read()

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:187: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


In [30]:
er_file

root hdmf.common.resources.ExternalResources at 0x140344390158944
Fields:
  entities: entities <class 'hdmf.common.resources.EntityTable'>
  keys: keys <class 'hdmf.common.resources.KeyTable'>
  object_keys: object_keys <class 'hdmf.common.resources.ObjectKeyTable'>
  objects: objects <class 'hdmf.common.resources.ObjectTable'>
  resources: resources <class 'hdmf.common.resources.ResourceTable'>

In [12]:
import datetime
from pynwb import TimeSeries, NWBHDF5IO
from pynwb.core import DynamicTable

from ndx_external_resources import ERNWBFile


nwbfile = ERNWBFile(
    session_description='session_description',
    identifier='identifier',
    session_start_time=datetime.datetime.now(datetime.timezone.utc)
)

container = TimeSeries(
    name='test_ts',
    data=[1, 2, 3],
    unit='meters',
    timestamps=[0.1, 0.2, 0.3],
)
nwbfile.add_acquisition(container)

table = DynamicTable(name='test_table', description='test table description')
table.add_column(name='test_col', description='test column description')
table.add_row(test_col='Mouse')

nwbfile.add_acquisition(table)

nwbfile.external_resources.add_ref(
    container=container,
    attribute='unit',
    key='meters',
    resource_name='SI_Ontology',
    resource_uri='',
    entity_id='5',
    entity_uri='',
)

nwbfile.external_resources.add_ref(
    container=table,
    attribute='test_col',
    key='Mouse',
    resource_name='NCBI_Taxonomy',
    resource_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi',
    entity_id='10090',
    entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=info&id=10090',
)

path = 'test.nwb'
with NWBHDF5IO(path, mode='w') as io:
    io.write(nwbfile)
    
with NWBHDF5IO(path, mode='r', load_namespaces=True) as io:
    read_nwbfile = io.read()
    read_container = read_nwbfile.acquisition['test_ts']
    read_table = read_nwbfile.acquisition['test_table']
    df=read_nwbfile.external_resources.get_object_resources(container=table['test_col'])
df

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:187: UserWarning: NWBExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))
/Users/mavaylon/Research/NWB/hdmf/src/hdmf/utils.py:629: FutureWarning: ExternalResources.__init__: Using positional arguments for this method is discouraged and will be deprecated in a future major release. Please use keyword arguments to ensure future compatibility.
  warnings.warn(msg, FutureWarning)
/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:187: UserWarning: NWBExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))
/Users/mavaylon/Research/NWB/hdmf/src/hdmf/utils.py:629: FutureWarning: ExternalResources.__init__: Using positional arguments for this method is discouraged and will be deprecated in a future major release. Please use keyword argument

,keys_idx,resource_idx,entity_id,entity_uri
0,1,1,10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
